# Chapter 3: RAG Part II: Chatting with Your Data

## Query Construction

As discussed earlier, RAG is an effective strategy to embed and retrieve relevant unstructured data from a vector store based on a query. But most data available for use in production apps is structured and typically stored in relational databases. In addition, unstructured data embedded in a vector store also contains structured metadata that possesses important information.

_Query construction_ is the process of transforming a natural language query into the query language of the database or data source you are interacting with.

### Text-to-SQL

SQL and relational databases are important sources of structured data, but they don’t interact directly with natural language. Although we can simply use the LLM to translate a user’s query to SQL queries, there is little margin for error.

Here are some useful strategies for effective text to SQL translations:

**_Database description_**

To ground SQL queries, an LLM must be provided with an accurate description   of the database. One common text-to-SQL prompt employs an idea reported in several papers: provide the LLM with a ```CREATE TABLE``` description for each table, including column names and types. We can also provide a few (for  instance, three) example rows from the table.

_**Few-shot examples**_

Feeding the prompt with few-shot examples of question-query matches can improve the query generation accuracy. This can be achieved by simply appending standard static examples in the prompt to guide the agent on how it should build queries based on questions.

Here’s a full code example:

1. Inspect the database

In [2]:
from langchain_community.tools import QuerySQLDatabaseTool
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_deepseek import ChatDeepSeek
from dotenv import load_dotenv


load_dotenv()

db = SQLDatabase.from_uri("sqlite:///chinook.db")
print(db.get_usable_table_names())

['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


2. Convert question to SQL query

In [9]:
llm = ChatDeepSeek(model="deepseek-chat", temperature=0)

def parse_query(query: str) -> str:
    """
    Parse the SQL query returned by write_query before sending it to execute_query.

    The query returned by write_query is originally in the form: SQLQuery: 'SQL statement'

    This function returns the SQL statement.
    """

    return query.split("SQLQuery: ")[1].strip()

write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDatabaseTool(db=db)
combined_chain = write_query | parse_query | execute_query

3. Run the chain

In [13]:
response  = combined_chain.invoke(input={"question": "How many employees are there?"})
print(response)

[(8,)]


In [17]:
response = combined_chain.invoke(input={"question": "Give me the names, birthdays, and hire dates of sales support agents"})
print(response)

[('Jane', 'Peacock', '1973-08-29 00:00:00', '2002-04-01 00:00:00'), ('Margaret', 'Park', '1947-09-19 00:00:00', '2003-05-03 00:00:00'), ('Steve', 'Johnson', '1965-03-03 00:00:00', '2003-10-17 00:00:00')]


We first convert the user’s query to a SQL query appropriate to the dialect of our database. Then we execute that query on our database. Note that executing arbitrary SQL queries on your database generated by an LLM from user input is dangerous in a production application.